<a href="https://colab.research.google.com/github/elonezra/deeplearningproject/blob/main/final_project_deep_learning_course.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Authors: Avi and Elon 🤗

In [81]:
# Importng necassary libraries
import pandas as pd
import numpy as np
import tensorflow.compat.v1 as tf
tf.compat.v1.disable_eager_execution()
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import warnings

In [82]:
df = pd.read_csv('BTC-USD.csv')
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
1,2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2,2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700
3,2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600
4,2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100


In [83]:
# We don't need the "Adj Close" column because this column is the same as the "Close" column,
# we are going to check it before earasing to make sure our assumption is true.
print(df['Adj Close'].count(), df['Close'].count()) # In this line we can see that the number of lines in both of the columns is equal
print(len(df["Close"] == df["Adj Close"])) # and in this line we compared between those columns and get as result that all lines are equal
# thus, we can remove "Asj Close" column
df = df.drop(['Adj Close'], axis = 1)
df.head()

2713 2713
2713


,Date,Open,High,Low,Close,Volume
0,2014-09-17,465.864014,468.174011,452.421997,457.334015,21056800
1,2014-09-18,456.859985,456.859985,413.104004,424.440002,34483200
2,2014-09-19,424.102997,427.834991,384.532013,394.795990,37919700
3,2014-09-20,394.673004,423.295990,389.882996,408.903992,36863600
4,2014-09-21,408.084991,412.425995,393.181000,398.821014,26580100


In [84]:
# now we want to change the 'Close' column to be 'Open_Close_Mean' column because,
# it's more common to predict the mean between the Open price and the Close price
df['Close'].rename('Open_Close_Mean')
df['Open_Close_Mean'] = (df['Open'] + df['Close']) / 2
df = df.drop(['Close'], axis=1)

In [85]:
# In the code below we will see that our dataframe uses an object column which is not good
# because we need a numeric dataframe so, we convert this column to int64 type
# by removing the '-' char
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2713 entries, 0 to 2712
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             2713 non-null   object 
 1   Open             2713 non-null   float64
 2   High             2713 non-null   float64
 3   Low              2713 non-null   float64
 4   Volume           2713 non-null   int64  
 5   Open_Close_Mean  2713 non-null   float64
dtypes: float64(4), int64(1), object(1)
memory usage: 127.3+ KB


In [86]:
int(df.iloc[1][0].replace('-', ''))

C:\Users\rahim\AppData\Local\Temp\ipykernel_15184\84532947.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  int(df.iloc[1][0].replace('-', ''))


20140918

In [87]:
df["Date"] = df["Date"].apply(lambda x: x.replace('-','')).apply(pd.to_numeric)

In [88]:
# Now, after we convert the "Date" column to numeric type, we can see the new data as request
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2713 entries, 0 to 2712
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             2713 non-null   int64  
 1   Open             2713 non-null   float64
 2   High             2713 non-null   float64
 3   Low              2713 non-null   float64
 4   Volume           2713 non-null   int64  
 5   Open_Close_Mean  2713 non-null   float64
dtypes: float64(4), int64(2)
memory usage: 127.3 KB


,Date,Open,High,Low,Volume,Open_Close_Mean
0,20140917,465.864014,468.174011,452.421997,21056800,461.599015
1,20140918,456.859985,456.859985,413.104004,34483200,440.649993
2,20140919,424.102997,427.834991,384.532013,37919700,409.449494
3,20140920,394.673004,423.295990,389.882996,36863600,401.788498
4,20140921,408.084991,412.425995,393.181000,26580100,403.453003


$\sigma = \sqrt(\mu_2) = \sqrt(1/n \sum(x_i - x')^2)$

$𝜇^{𝑙,𝑗}= \frac{1}{m} ∑_{𝑖=0}^{𝑚}{𝑧_𝑖^{𝑙,𝑗}}$ 

$𝜎^{𝑙,𝑗}=\sqrt{∑_{𝑖=0}^𝑚(𝑧_𝑖^{𝑙,𝑗}−𝜇^(𝑙,𝑗) )^2 }$

$\overline{𝑧_𝑖^{𝑙,𝑗}} ̂= \frac{𝑧_𝑖^{𝑙,𝑗}−𝜇^{𝑙,𝑗}}{𝜎^{𝑙,𝑗}+𝜖} $


In [89]:
from sklearn.model_selection import train_test_split

X = df.drop(['Open_Close_Mean'], axis=1)
y = df['Open_Close_Mean']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training set size:", len(X_train))
print("Testing set size:", len(X_test))


Training set size: 2170
Testing set size: 543


In [90]:
# Convert the dataframes to numpy arrays
train_np = X_train.values
test_np = X_test.values

# Convert the series to numpy arrays
y_train_np = y_train.values.reshape(-1, 1)
y_test_np = y_test.values.reshape(-1, 1)

# Create TensorFlow placeholders
train_placeholder = tf.placeholder(tf.float32, shape=train_np.shape)
test_placeholder = tf.placeholder(tf.float32, shape=test_np.shape)
y_train_placeholder = tf.placeholder(tf.float32, shape=y_train_np.shape)
y_test_placeholder = tf.placeholder(tf.float32, shape=y_test_np.shape)

# Define the normalization function
def normalize(x, mean, variance):
    return (x - mean) / tf.sqrt(variance)

# Normalize the data
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Compute mean and variance from training data
    train_mean, train_variance = sess.run(tf.nn.moments(train_placeholder, axes=[0]), feed_dict={train_placeholder: train_np})
    
    # Normalize training data
    train_normalized = sess.run(normalize(train_placeholder, train_mean, train_variance), feed_dict={train_placeholder: train_np})
    
    # Normalize test data using mean and variance from training data
    test_normalized = sess.run(normalize(test_placeholder, train_mean, train_variance), feed_dict={test_placeholder: test_np})
    
    # Normalize y_train and y_test similarly
    y_train_mean, y_train_variance = sess.run(tf.nn.moments(y_train_placeholder, axes=[0]), feed_dict={y_train_placeholder: y_train_np})
    y_train_normalized = sess.run(normalize(y_train_placeholder, y_train_mean, y_train_variance), feed_dict={y_train_placeholder: y_train_np})
    y_test_normalized = sess.run(normalize(y_test_placeholder, y_train_mean, y_train_variance), feed_dict={y_test_placeholder: y_test_np})

# Convert the normalized numpy arrays back to pandas dataframes
X_train_normalized = pd.DataFrame(train_normalized, columns=X_train.columns)
X_test_normalized = pd.DataFrame(test_normalized, columns=X_test.columns)

# Convert the normalized numpy arrays back to pandas series
y_train_normalized = pd.Series(y_train_normalized.flatten(), index=y_train.index)
y_test_normalized = pd.Series(y_test_normalized.flatten(), index=y_test.index)

print("Training set size:", len(X_train_normalized))
print("Testing set size:", len(X_test_normalized))


Training set size: 2170
Testing set size: 543


In [91]:
# to extract the numebr of features in out dataset we need to index to the
# second element in the shape of the dataframe which is the number of columns
# in our dataframe and -1 for the features without the label
features = df.shape[1] - 1
x = tf.placeholder(tf.float32, [None, features])
y = tf.placeholder(tf.float32, [None, 1])
W = tf.Variable(tf.zeros([features,1]))
b = tf.Variable(tf.zeros([1]))
pred = tf.matmul(x,W) + b
loss = tf.reduce_mean(tf.pow(pred - y, 2))
update = tf.train.GradientDescentOptimizer(0.001).minimize(loss)
data_x = X_train_normalized
data_y = y_train_normalized.values.reshape(-1, 1)
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for i in range(0,100000):
    sess.run(update, feed_dict = {x:data_x, y:data_y})
    if i % 10000 == 0:
        pass
        # print('Iteration:' , i , ' W:' , sess.run(W) , ' b:' , sess.run(b), ' loss:', loss.eval(session=sess, feed_dict = {x:data_x, y:data_y}))


In [92]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

b = tf.cast(b, tf.float32)  # Ensure b is float32
X_test_np = X_test_normalized.to_numpy().astype('float32')  # Convert X_test to numpy array of type float32

# Compute y_pred
y_pred_tensor = tf.matmul(X_test_np, W) + b  # Now X_test, W, and b are of the same type
y_pred = sess.run(y_pred_tensor)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test_normalized, y_pred))

# Calculate the range of the target variable
range_y = np.max(y_test_normalized) - np.min(y_test_normalized)

# Calculate NRMSE
nrmse = rmse / range_y

# Assuming y_test are the true target values and y_pred are the predicted target values
r2 = r2_score(y_test_normalized, y_pred)

print("R-squared: ", r2)

r2_percentage = r2 * 100

print("R-squared (%): ", r2_percentage)

print("NRMSE: ", nrmse)


R-squared:  0.99988599280659
R-squared (%):  99.988599280659
NRMSE:  0.0026836244


In [93]:
%load_ext tensorboard
# Define the number of features
features = df.shape[1] - 1

# Define placeholders
x = tf.placeholder(tf.float32, [None, features])
y = tf.placeholder(tf.float32, [None, 1])

# Define weights and bias
W1 = tf.Variable(tf.random_normal([features, 10]))
b1 = tf.Variable(tf.zeros([10]))
W2 = tf.Variable(tf.random_normal([10, 1]))
b2 = tf.Variable(tf.zeros([1]))

# Define the model
hidden_layer = tf.nn.relu(tf.matmul(x, W1) + b1)
pred = tf.matmul(hidden_layer, W2) + b2

# Define the loss function (Mean Squared Error)
loss = tf.reduce_mean(tf.square(pred - y))

# Define the optimizer (Gradient Descent)
optimizer = tf.train.GradientDescentOptimizer(0.00001).minimize(loss)

# Initialize variables
init = tf.global_variables_initializer()

# Reshape y_train and y_test to be 2D arrays
y_train = y_train_normalized.values.reshape(-1, 1)
y_test = y_test_normalized.values.reshape(-1, 1)

# Start a Session
with tf.Session() as sess:
    sess.run(init)

    # Train the model
    for epoch in range(10000):
        _, c = sess.run([optimizer, loss], feed_dict={x: X_train_normalized, y: y_train})

        # Print loss every 1000 epochs
        if epoch % 1000 == 0:
            print("Epoch:", '%04d' % (epoch), "cost=", "{:.9f}".format(c))

    # Make predictions
    y_pred = sess.run(pred, feed_dict={x: X_test_normalized})

# Calculate R-Squared score
r2 = r2_score(y_test, y_pred)
print("R-Squared: ", r2)

r2_percentage = r2 * 100

print("R-squared (%): ", r2_percentage)

print("NRMSE: ", nrmse)


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Epoch: 0000 cost= 9.177856445
Epoch: 1000 cost= 6.758913040
Epoch: 2000 cost= 5.123683453
Epoch: 3000 cost= 4.035577297
Epoch: 4000 cost= 3.239751816
Epoch: 5000 cost= 2.634120464
Epoch: 6000 cost= 2.175396919
Epoch: 7000 cost= 1.832857132
Epoch: 8000 cost= 1.556816816
Epoch: 9000 cost= 1.335294962
R-Squared:  -0.8342292249812899
R-squared (%):  -83.422922498129
NRMSE:  0.0026836244
